In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pypsa

In [ ]:
n = pypsa.Network()

eta_el = 0.1
eta_dh = 0.8
eta_direct = 0.95

el_capital_cost = 1000 # EUR/kW
dh_capital_cost = 1000 # EUR/kW

eta = eta_direct / eta_dh

n.add(
    "Link",
    "geothermal CHP electricity",
    bus0='geothermal well',
    bus1='AC bus',
    efficiency=eta_el * (1. + eta),
    capital_cost=el_capital_cost * eta_el,
    p_max_pu=0.5,
)
n.add(
    "Link",
    "geothermal CHP district heat",
    bus0='geothermal well',
    bus1='DH bus',
    efficiency=eta_dh * (1 + eta),
    capital_cost=dh_capital_cost * eta_dh,
    p_max_pu=eta / 2.,
)

# Plus additional constraints

# 1. p_nom of both links equal                                           -> CHP operation
# 2. p_el > alpha p_dh                                                   -> backpressure limit
# 3. p_el = p_nom * (eta_direct - (p_dh / p_nom)) / (eta_direct - 1)     -> iso fuel

# recall p_nom = p_nom_el = p_nom_dh






In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(10, 6))

ax.plot([0, 1], [1, 1,], color='red', label='ORC capacity', linewidth=2)


eta_dh = 0.8 # efficiency when heating with ORC waste heat
eta_direct = 0.95 # efficiency when heating directly with geothermal heat

e = eta_direct / eta_dh

ax.plot([1, e], [1, 0], color='blue', label='Iso fuel line', linewidth=2)

s = 0.2

m = 1 / (1 - e)
b = e / (e - 1)

x = b / (s - m)
y = x * s

ax.plot([0, x], [0, y], color='green', label='Backpressure Limit', linewidth=2, linestyle='dashed')
ax.axvline(0, color='black', linewidth=1)
ax.axhline(0, color='black', linewidth=1)

ax.set_xlabel("Heat Generation")
ax.set_ylabel("Power Generation")

fill_x = np.linspace(0, e, 100)
ax.fill_between(
    fill_x,
    np.zeros_like(fill_x),
    np.minimum(np.ones_like(fill_x), m*fill_x + b),
    color='red', alpha=0.2)

ax.text(1.2, 0.02, "A")
ax.text(1.01, 1.01, "B")

ax.legend()

ax.grid('dashed')
ax.set_ylim(-0.05, 1.1)
ax.set_xlim(-0.05, 1.4)

plt.show()